# 图像特征—SIFT尺度不变特征变换

### 1.1 图像尺度空间
- 在一定的范围内，无论物体是大还是小，人眼都可以分辨出来，然后计算机要具有相同的能力却很难，所以要让机器能够对物体在不同尺度下有一个统一的认知，就需要考虑图像在不同尺度下都存在的特点。

### 尺度空间的获取通常使用高斯模糊来实现

![title](ImgFeature/sift1.png)
![title](ImgFeature/sift2.png)
- 不同 $\sigma$ 的高斯函数决定了对图像的平滑程度，越大的 $\sigma$ 值对应的图像越模糊。

### 1.2 多分辨率金字塔
![title](ImgFeature/sift3.png)

### 1.3 高斯差分金字塔(DoG)
![title](ImgFeature/sift4.png)
![title](ImgFeature/sift5.png)

### 1.4 DoG空间极值检测
- 为了寻找尺度空间的极值点，每个像素点要和其图像域(同一尺度空间)和尺度域(相邻的尺度空间)的所有相邻点进行比较，当其大于(或者小于)所有相邻点时，该点就是极值点。如下图所示，中间的检测点要和其所在图像的3*3邻域8个像素点，以及其相邻的上下两层3*3邻域18个像素点，共26个像素点进行比较。


![title](ImgFeature/sift6.png)

### 1.5 关键点的精确定位
- 这些候选关键点是DoG空间的局部极值点，而且这些极值点均为离散的点，精确定位极值点的一种方法是对尺度空间DoG函数进行曲线拟合，计算其极值点，从而实现关键点的精确定位。

![title](ImgFeature/sift7.png)

### 1.6 消除边界响应
![title](ImgFeature/sift8.png)

### 1.7 特征点的主方向
![title](ImgFeature/sift9.png)

### 2.1 生成特征描述
- 在完成关键点的梯度计算后，使用直方图统计领域内像素的梯度和方向。
![title](ImgFeature/sift10.png)
- 为了保证特征矢量的旋转不变性，要以特征点为中心，在附近邻域内将坐标轴旋转 $\theta$ 角度，即将坐标轴旋转为特征点的主方向。
![title](ImgFeature/sift11.png)

- 旋转之后的主方向为中心取8*8的窗口，求每个像素的梯度幅值和方向，箭头方向代表梯度方向，长度代表梯度幅值，然后利用高斯窗口对其进行加权计算，最后在每个4*4的小块上绘制8个方向的梯度直方图，计算每个梯度直方图的累加值，即可形成一个种子点，即每个特征由4个种子点组成，每个种子点有8个方向的向量信息。

![title](ImgFeature/sift12.png)

- 论文中建议对每个关键点使用4*4共16个种子点来描述，这样一个关键点就会产生128维的SIFT特征向量。

![title](ImgFeature/sift13.png)

## OpenCV SIFT函数

In [1]:
import cv2  #opencv 读取进来为BGR格式
import matplotlib.pyplot as plt
import numpy as np

#一个魔法指令，不需要采用plt.show()显示图像
%matplotlib inline   

print(cv2.__version__)   # 3.4.1.15 pip install opencv-python ==3.4.1.15  pip install opencv-contrib-python==3.4.1.15

4.4.0


In [2]:
def cv_show(name,img):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
img = cv2.imread('images/test2.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

cv_show('gray',gray)

In [4]:
# 得到特征点
sift = cv2.xfeatures2d.SIFT_create()
kp = sift.detect(gray,None)

img = cv2.drawKeypoints(gray,kp,img)
cv_show('keyPoint',img)

In [5]:
# 计算特征
kp, des = sift.compute(gray, kp)
print(np.array(kp).shape)

(593,)


In [6]:
print(des.shape)

(593, 128)


In [7]:
print(des[0])

[ 18. 133.  26.   0.   0.   0.   6.  18.   7. 140.  77.   0.   0.   0.
   1.   3.  24. 140.  42.   0.   0.   0.   0.   0.   9.  18.   1.   0.
   2.   3.   1.   0.  17.  13.   2.   0.   0.  45. 140.  14.  32.  79.
  35.   6.   2.  53.  69.  17. 140. 140.  12.   0.   0.   1.  12.  51.
  72.  17.   1.   3.   2.   0.   0.   5.   1.   1.   0.   0.   0.  80.
 140.   3.   7.   1.   1.   0.   1. 134. 140.  39.  96.   2.   0.   0.
   0.   5. 103. 140.  33.   3.   3.   2.   1.   0.   0.  19.   6.  11.
   0.   0.   0.   3.   7.   1.   1.   3.   0.   0.   0.  11.  35.   3.
   3.   3.   1.   0.   0.   1.  18.  16.   1.   4.   4.   0.   0.   0.
   0.   1.]
